In [75]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


from urllib.parse import urljoin

def fetch_website_links(url):
    """
    Return the links on the website at the given url.
    Converts relative URLs to absolute URLs.
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = []
    
    for link in soup.find_all("a"):
        href = link.get("href")
        if href:
            # Convert relative URLs to absolute URLs
            absolute_url = urljoin(url, href)
            links.append(absolute_url)
    
    return links


In [76]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
#from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [77]:
# Load environment variables
load_dotenv()

True

In [78]:
# Initialize Groq client
groq = OpenAI(
    api_key=os.getenv('GROQ_API_KEY'),
    base_url='https://api.groq.com/openai/v1'
)

In [80]:
links = fetch_website_links("https://arstechnica.com/")
links

['https://arstechnica.com/#main',
 'https://arstechnica.com/',
 'https://arstechnica.com/civis/',
 'https://arstechnica.com/subscribe/',
 'https://arstechnica.com/search/',
 'https://arstechnica.com/ai/',
 'https://arstechnica.com/information-technology/',
 'https://arstechnica.com/cars/',
 'https://arstechnica.com/culture/',
 'https://arstechnica.com/gaming/',
 'https://arstechnica.com/health/',
 'https://arstechnica.com/tech-policy/',
 'https://arstechnica.com/science/',
 'https://arstechnica.com/security/',
 'https://arstechnica.com/space/',
 'https://arstechnica.com/gadgets/',
 'https://arstechnica.com/features/',
 'https://arstechnica.com/reviews/',
 'https://arstechnica.com/ai/',
 'https://arstechnica.com/information-technology/',
 'https://arstechnica.com/cars/',
 'https://arstechnica.com/culture/',
 'https://arstechnica.com/gaming/',
 'https://arstechnica.com/health/',
 'https://arstechnica.com/tech-policy/',
 'https://arstechnica.com/science/',
 'https://arstechnica.com/securi

In [63]:
link_system_prompt = """
""You are a tech news curator. Analyze the provided links and identify groundbreaking technology news.

Focus on:
- AI/ML breakthroughs and research
- Major product launches (AI tools, new platforms)
- Significant corporate tech announcements
- Emerging technologies that could reshape industries
- Policy/regulation changes affecting tech

Ignore:
- Basic tech tips and how-tos
- Product reviews of consumer gadgets
- Gaming news (unless breakthrough technology)
- General navigation/homepage links
- Privacy policies, terms of service

You MUST respond with ONLY valid JSON, no other text. Use this exact format:

{
    "links": [
        {"type": "AI Advancement", "url": "https://example.com/article1"},
        {"type": "Corporate Announcement", "url": "https://example.com/article2"}
    ]
}

IMPORTANT: Return ONLY the JSON object, nothing else."""

In [64]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a tech newsletter that includes groundbreaking technews, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [65]:
print(get_links_user_prompt("https://timesofindia.indiatimes.com/technology"))


Here is the list of links on the website https://timesofindia.indiatimes.com/technology -
Please decide which of these are relevant web links for a tech newsletter that includes groundbreaking technews, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://timesofindia.indiatimes.com/
https://timesofindia.indiatimes.com/us
https://timesofindia.indiatimes.com/
https://navbharattimes.indiatimes.com/
https://marathi.indiatimes.com/
https://vijaykarnataka.com/
https://tamil.samayam.com/
https://bangla.indiatimes.com/
https://malayalam.samayam.com/
https://telugu.samayam.com/
https://www.iamgujarat.com/
https://timesofindia.indiatimes.com/weather
https://timesofindia.indiatimes.com/technology
https://timesofindia.indiatimes.com
https://timesofindia.indiatimes.com/technology
https://timesofindia.indiatimes.com/technology/tech-tips
https://timesofindia.indiatimes.com/technology/tech-news
https:/

In [83]:
def select_relevant_links(url):
    response = groq.chat.completions.create(
        model='llama-3.3-70b-versatile',
        messages=[
            {'role': 'system', 'content': link_system_prompt},
            {'role': 'user', 'content': get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"},
        temperature=0
    )
    
    result = response.choices[0].message.content
    return json.loads(result)

In [84]:
result = select_relevant_links('https://arstechnica.com/')
print(json.dumps(result, indent=2))

{
  "links": [
    {
      "type": "AI Advancement",
      "url": "https://arstechnica.com/ai/2025/11/openai-walks-a-tricky-tightrope-with-gpt-5-1s-eight-new-personalities/"
    },
    {
      "type": "AI Research",
      "url": "https://arstechnica.com/ai/2025/11/study-finds-ai-models-store-memories-and-logic-in-different-neural-regions/"
    },
    {
      "type": "Corporate Announcement",
      "url": "https://arstechnica.com/gadgets/2025/11/microsoft-releases-update-fixing-update-for-update-eligible-windows-10-pcs/"
    },
    {
      "type": "Emerging Technology",
      "url": "https://arstechnica.com/science/2025/11/quantum-roundup-lots-of-companies-announcing-new-tech/"
    },
    {
      "type": "Tech Policy",
      "url": "https://arstechnica.com/tech-policy/2025/11/openai-fights-order-to-hand-over-20-million-private-chatgpt-conversations/"
    },
    {
      "type": "Space Exploration",
      "url": "https://arstechnica.com/space/2025/11/with-another-record-broken-the-worlds-

# Second Step 

In [85]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [86]:
print(fetch_page_and_all_relevant_links("https://arstechnica.com/"))

## Landing Page:

Ars Technica - Serving the Technologist since 1998. News, reviews, and analysis.

Skip to content
Ars Technica home
Sections
Forum
Subscribe
Search
AI
Biz & IT
Cars
Culture
Gaming
Health
Policy
Science
Security
Space
Tech
Feature
Reviews
AI
Biz & IT
Cars
Culture
Gaming
Health
Policy
Science
Security
Space
Tech
Forum
Subscribe
Theme
HyperLight
Day & Night
Dark
System
Sign In
Ars Technica homepage
Grid
Classic
Grid
List
Neutron Star
Classic
Grid
List
Neutron Star
grid
          settings
Featured
framework laptop 16, take two
Review: New Framework Laptop 16 takes a fresh stab at the upgradeable laptop GPU
New components make it more useful and powerful but no less odd.
Andrew Cunningham
–
11/12/2025
|
48
OpenAI walks a tricky tightrope with GPT-5.1’s eight new personalities
New controls attempt to please critics on both sides with a balance between bland and habit-forming.
Benj Edwards
–
11/12/2025
|
21
With another record broken, the world’s busiest spaceport keeps gett

In [99]:
newsletter_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a Tech news website
and creates a short newsletter about the groundbreaking tech news for prospective students, investors and development enthusiasts.
Respond in markdown without code blocks.
Include details of news whataboutsand future translations or impacts for each news if you have the information.
Analyze the identified groundbreaking technology news.

Focus on:
- AI/ML breakthroughs and research
- Major product launches (AI tools, new platforms)
- Significant corporate tech announcements
- Emerging technologies that could reshape industries
- Policy/regulation changes affecting tech

Ignore:
- Basic tech tips and how-tos
- Product reviews of consumer gadgets
- Gaming news (unless breakthrough technology)
- General navigation/homepage links
- Privacy policies, terms of service
"""


In [100]:
def get_newsletter_user_prompt(title, url):
    user_prompt = f"""
You are looking at a news called: {title}
Here are the contents of its landing page and other relevant pages;
use this information to build a detailed newsletter in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [101]:
get_newsletter_user_prompt("Today's Ground Breaking Tech News Letter", "https://arstechnica.com/")

"\nYou are looking at a news called: Today's Ground Breaking Tech News Letter\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a detailed newsletter in markdown without code blocks.\n\n\n## Landing Page:\n\nArs Technica - Serving the Technologist since 1998. News, reviews, and analysis.\n\nSkip to content\nArs Technica home\nSections\nForum\nSubscribe\nSearch\nAI\nBiz & IT\nCars\nCulture\nGaming\nHealth\nPolicy\nScience\nSecurity\nSpace\nTech\nFeature\nReviews\nAI\nBiz & IT\nCars\nCulture\nGaming\nHealth\nPolicy\nScience\nSecurity\nSpace\nTech\nForum\nSubscribe\nTheme\nHyperLight\nDay & Night\nDark\nSystem\nSign In\nArs Technica homepage\nGrid\nClassic\nGrid\nList\nNeutron Star\nClassic\nGrid\nList\nNeutron Star\ngrid\n          settings\nFeatured\nframework laptop 16, take two\nReview: New Framework Laptop 16 takes a fresh stab at the upgradeable laptop GPU\nNew components make it more useful and powerful but no less odd.\nAndrew Cunn

In [102]:
def create_newsletter(title, url):
    response = groq.chat.completions.create(
        model = 'llama-3.3-70b-versatile',
        messages= [
            {'role':'system','content':newsletter_system_prompt},
            {'role':'user','content':get_newsletter_user_prompt(title,url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [103]:
create_newsletter('Todays Ground Breaking Tech-News Letter', "https://arstechnica.com/")

## Groundbreaking Tech News Letter
### Introduction
Welcome to our latest edition of the Groundbreaking Tech News Letter, where we bring you the most significant and exciting developments in the world of technology. In this issue, we will be covering the latest advancements in AI/ML research, major product launches, and significant corporate tech announcements that are shaping the future of the industry.

### AI/ML Breakthroughs and Research
#### OpenAI's GPT-5.1: A New Era in Conversational AI
OpenAI has released GPT-5.1 Instant and GPT-5.1 Thinking, two updated versions of its flagship AI models now available in ChatGPT. The new models boast warmer, more conversational, and better instruction-following capabilities. This release comes after complaints about the previous models being excessively cheerful and sycophantic, as well as controversy surrounding the modification of the default GPT-5 output style. The new models aim to strike a balance between being engaging and informative, while also addressing concerns about their potential impact on users.

#### Isolating Memorization from Problem-Solving in AI Neural Networks
Researchers have made a significant breakthrough in understanding how AI neural networks work, isolating memorization from problem-solving. This discovery has shown that basic arithmetic ability lives in the memorization pathways, which could have major implications for the development of more advanced AI systems. By understanding how AI models learn and process information, researchers can create more efficient and effective algorithms, leading to significant advancements in areas such as natural language processing, computer vision, and decision-making.

### Major Product Launches
#### Framework Laptop 16: A New Take on the Upgradeable Laptop
The Framework Laptop 16 has been released, offering a fresh take on the upgradeable laptop concept. With new components and a more powerful GPU, this laptop is more useful and powerful than its predecessors, but still maintains its unique oddity. This release is significant, as it shows that manufacturers are willing to experiment with new designs and ideas, which could lead to more innovative and sustainable products in the future.

### Significant Corporate Tech Announcements
#### Microsoft Releases Update-Fixing Update for Windows 10 PCs
Microsoft has released an update-fixing update for Windows 10 PCs, addressing a bug that was keeping some PCs from enrolling in the company's ESU program. This move is important, as it shows that Microsoft is committed to supporting its older operating systems and ensuring that users have access to the latest security updates and features.

### Emerging Technologies that Could Reshape Industries
#### Space Exploration: The World's Busiest Spaceport Keeps Getting Busier
The world's busiest spaceport has broken another record, with an increasing number of rocket launches and cargo being carried into orbit. This trend is significant, as it shows that space exploration is becoming more accessible and affordable, which could lead to major breakthroughs in areas such as satellite technology, space tourism, and asteroid mining.

### Policy/Regulation Changes Affecting Tech
#### NASA Postpones Launch due to Space Weather Conditions
NASA has postponed a launch due to space weather conditions, highlighting the importance of considering the impact of space weather on satellite launches and space exploration. This move is significant, as it shows that regulatory bodies are taking a cautious approach to space exploration, prioritizing safety and minimizing the risk of damage to equipment and personnel.

### Conclusion
In conclusion, this week's Groundbreaking Tech News Letter has highlighted some of the most significant developments in the world of technology, from AI/ML research and major product launches to significant corporate tech announcements and emerging technologies that could reshape industries. As the tech landscape continues to evolve, it's exciting to think about the potential implications of these advancements and how they could shape the future of our world.